In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
import subprocess
from subprocess import Popen, PIPE
import seaborn as sns

# Chartevents further wrangling

In [2]:
## Extract features from chart_events (still too big to upload into python)

In [3]:
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/icu'
items = pd.read_csv(path+'/d_items.csv')

In [4]:
items.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
2,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
3,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN


In [5]:
##Name of features in table
features = ['Heart Rate', 'O2 saturation pulseoxymetry','Respiratory Rate', 'ZC Reactive Protein (CRP)'
            'Temperature Fahrenheit', 'Temperature Celsius',
            'Non Invasive Blood Pressure systolic', 'Arterial Blood Pressure systolic',
           'Arterial Blood Pressure mean', 'Arterial Blood Pressure diastolic',
           'Non Invasive Blood Pressure diastolic','Non Invasive Blood Pressure diastolic', 
           'EtCO2','Arterial Base Excess',
           'Inspired O2 Fraction', 'Arterial CO2 Pressure',
            'ART %O2 saturation (PA Line)', 'Direct Bilirubin', 'BUN', 'BUN_ApacheIV'
            'AST', 'PH', 'Chloride (serum)',
            'Glucose (serum)', 'Magnesium',
            'Alkaline Phosphate','Creatinine (whole blood)','Hematocrit (serum)',
            'Ammonia', 'Lactic Acid',
            'Hemoglobin','Total Bilirubin', 'Bilirubin_ApacheIV','Troponin-T', 'PTT', 'WBC',
            'ZFibrinogen', 'Platelet Count', 'Potassium (serum)', 'ZPotassium (serum)', 'Urine output_ApacheIV',
            'GU Irrigant/Urine Volume Out GU Irrigant/Urine Volume', 'Sodium (serum)', 'WBC', 'WBC_ApacheIV',
            'Creatinine (serum)', 'Platelet Count', 'GCS - Eye Opening', 'GCS - Verbal Response',
            'GCS - Motor Response', 'C Reactive Protein (CRP)', 'Absolute Neutrophil Count',
            'D-Dimer','LDH', 'OxygenScore_ApacheIV'
           ]

In [6]:
#extract itemids and save
features_analysis = items[items['label'].isin(features)]
features_analysis = pd.DataFrame(features_analysis['itemid'])
features_analysis.to_csv('1.0/small/icu/chartevents_features.csv', index = False)

In [7]:
#extract features from chartevents
request = 'awk -F, \'FNR==NR{a[$1]++;next}a[$6]\' chartevents_features.csv chartevents.csv > chartevents_filtered.csv'

# Create cohort (procedure + diagnosis + cxr available)

In [8]:
#Create the positive and negative cohorts
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
icustays = pd.read_csv(path + '/icu/icustays.csv', parse_dates = ['intime', 'outtime'])
admissions = pd.read_csv(path + 'core/admissions.csv', parse_dates = ['admittime', 'dischtime', 'deathtime'])
cxr = pd.read_csv(path + 'ids/mimic-cxr-2.0.0-metadata.csv', parse_dates = ['StudyDate'])
procedures = pd.read_csv(path + '/hosp/procedures_icd.csv', parse_dates = ['chartdate'])
diagnosis = pd.read_csv(path + '/hosp/diagnoses_icd.csv')
dx_icd_codes = pd.read_csv(path + 'small/hosp/icd_diagnosis.txt')
patients = pd.read_csv(path + 'core/patients.csv', parse_dates = ['dod'])
hadm_ids = pd.read_csv(path + 'ids/hadm_ids_all.csv')

In [9]:
##get the earliest time of ventilation for patients who were ventilated
ventilations = procedures[procedures['icd_code'].isin([
    '5A1935Z','5A1945Z','5A1955Z', '9670', '9671', 
    '9672', '9604','0BH17EZ' ,'0BH18EZ'])]

pos_outcome = ventilations.loc[ventilations.groupby('hadm_id').chartdate.idxmin(),[
                    'subject_id', 'hadm_id','chartdate']]

##get those with a resp diagnosis
diagnosis_resp = diagnosis[diagnosis['icd_code'].isin(dx_icd_codes['icd_code'])]
##merge to get inner join
pos_outcome = pos_outcome.merge(diagnosis_resp[['hadm_id','subject_id']], on = ['hadm_id','subject_id'])

In [10]:
##get the time of death for patients
admissions_dod = admissions[admissions['deathtime'].notna()]
admissions_dod.rename(columns = {'deathtime':'chartdate'}, inplace = True)
admissions_dod.merge(diagnosis_resp[['hadm_id','subject_id']], on = ['hadm_id','subject_id'])

##append this to the ventilation outcomes
pos_outcome = pos_outcome.append(admissions_dod[['subject_id', 'hadm_id','chartdate']])

/tmp/ipykernel_420924/1860779672.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissions_dod.rename(columns = {'deathtime':'chartdate'}, inplace = True)
/tmp/ipykernel_420924/1860779672.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pos_outcome = pos_outcome.append(admissions_dod[['subject_id', 'hadm_id','chartdate']])


In [11]:
##groupby admission and pick the earliest outcome i.e. remove death after ventilation
pos_outcome = pos_outcome.loc[pos_outcome.groupby('hadm_id').chartdate.idxmin(),['subject_id', 'hadm_id','chartdate']]
#create observation window for cxr
pos_outcome['obs_window_end'] = pos_outcome['chartdate'] - timedelta(days = 0.5)
pos_outcome['obs_window_start'] = pos_outcome['chartdate'] - timedelta(days = 5)
pos_outcome['outcome'] = 1

In [12]:
pos_outcome.head()

,subject_id,hadm_id,chartdate,obs_window_end,obs_window_start,outcome
91946,14046553,20000094,2150-03-03 09:21:00,2150-03-02 21:21:00,2150-02-26 09:21:00,1
0,16003661,20001305,2178-03-25 00:00:00,2178-03-24 12:00:00,2178-03-20 00:00:00,1
2,14577567,20001361,2143-05-04 00:00:00,2143-05-03 12:00:00,2143-04-29 00:00:00,1
3,10492653,20001729,2131-01-29 00:00:00,2131-01-28 12:00:00,2131-01-24 00:00:00,1
4,18834767,20002519,2155-02-10 00:00:00,2155-02-09 12:00:00,2155-02-05 00:00:00,1


In [13]:
##get the negative outcome group
neg_hadm_ids = set(diagnosis_resp['hadm_id']) - set(pos_outcome['hadm_id'])
neg_hadm_ids = pd.DataFrame(neg_hadm_ids, columns = ['hadm_id'])
neg_outcome = neg_hadm_ids.merge(icustays, left_on = 'hadm_id', right_on = 'hadm_id')[['hadm_id','subject_id', 'intime']]
neg_outcome = neg_outcome.loc[neg_outcome.groupby('hadm_id').intime.idxmin()]
neg_outcome.rename(columns = {'intime':'chartdate'}, inplace = True)
#create observation window for cxr
neg_outcome['obs_window_start'] = neg_outcome['chartdate'] - timedelta(days = 0.5)
neg_outcome['obs_window_end'] = neg_outcome['chartdate'] + timedelta(days = 4)
neg_outcome['outcome'] = 0

In [14]:
neg_outcome.head()

,hadm_id,subject_id,chartdate,obs_window_start,obs_window_end,outcome
5530,20001395,16679562,2180-04-24 00:08:00,2180-04-23 12:08:00,2180-04-28 00:08:00,0
5554,20001770,10117812,2117-01-25 20:47:00,2117-01-25 08:47:00,2117-01-29 20:47:00,0
5677,20003491,11540283,2197-12-18 06:10:00,2197-12-17 18:10:00,2197-12-22 06:10:00,0
5949,20007037,16147352,2129-11-14 14:43:18,2129-11-14 02:43:18,2129-11-18 14:43:18,0
5993,20007567,12351713,2166-08-02 14:59:20,2166-08-02 02:59:20,2166-08-06 14:59:20,0


In [15]:
##create cohort
outcome = pos_outcome.append(neg_outcome)
outcome.shape

/tmp/ipykernel_420924/336775330.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outcome = pos_outcome.append(neg_outcome)


(26426, 6)

In [16]:
outcome.head()

,subject_id,hadm_id,chartdate,obs_window_end,obs_window_start,outcome
91946,14046553,20000094,2150-03-03 09:21:00,2150-03-02 21:21:00,2150-02-26 09:21:00,1
0,16003661,20001305,2178-03-25 00:00:00,2178-03-24 12:00:00,2178-03-20 00:00:00,1
2,14577567,20001361,2143-05-04 00:00:00,2143-05-03 12:00:00,2143-04-29 00:00:00,1
3,10492653,20001729,2131-01-29 00:00:00,2131-01-28 12:00:00,2131-01-24 00:00:00,1
4,18834767,20002519,2155-02-10 00:00:00,2155-02-09 12:00:00,2155-02-05 00:00:00,1


In [17]:
#filter out the lateral views
cxr = cxr[(cxr['ViewPosition'] != 'LATERAL') & (cxr['ViewPosition'] != 'LL')]

In [18]:
#filter cxr for our subjects
cxr_filtered = cxr[cxr['subject_id'][(cxr['ViewPosition'] != 'LATERAL') & 
                                     (cxr['ViewPosition'] != 'LL')].isin(outcome['subject_id'])]

In [19]:
# merge with outcomes
cxr_filtered = cxr_filtered.merge(outcome, on = 'subject_id')

In [20]:
## filter for cxr in the obs window
cxr_filtered = cxr_filtered[(cxr_filtered['obs_window_end'] >= cxr_filtered['StudyDate']) & 
                             (cxr_filtered['obs_window_start'] <= cxr_filtered['StudyDate'])]
cxr_use = cxr_filtered.loc[cxr_filtered.groupby('subject_id').StudyDate.idxmax()]

In [21]:
##save the cohort data
outcome = cxr_use[['hadm_id','subject_id', 'chartdate','outcome','obs_window_start', 'obs_window_end']]
outcome_cohort = cxr_use[['hadm_id','subject_id','outcome','dicom_id']]
outcome_cohort.to_csv(path + 'small/cohort_use.csv', index = False)
print(outcome_cohort.shape[0], 
      outcome_cohort[outcome_cohort['outcome']==1].shape[0], 
      outcome_cohort[outcome_cohort['outcome']==0].shape[0])
outcome_cohort.head()


4491 2635 1856


,hadm_id,subject_id,outcome,dicom_id
20,26184834,10001884,1,9fd47edd-07087209-b901811e-3e9e5f50-f382f611
33,28662225,10002428,1,e8c44648-ff02beea-3d5ff638-dec79b01-7df71a69
138,22774359,10003019,0,c25b9023-9cd26369-dbe2b67b-c250ef52-b772137e
174,23559586,10003400,1,5a907c47-9d944216-c8477dd2-95d08914-13239bec
237,28477357,10007795,0,0aaf1dce-44cd06d6-29ad91bd-d70fe036-e3823187


# demographic features

In [22]:
#extract demographic features
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
patients = pd.read_csv(path + 'core/patients.csv', parse_dates = ['anchor_year'])
admissions = pd.read_csv(path + 'core/admissions.csv', parse_dates = ['admittime'])

In [23]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [24]:
#merge with admissions and patients to get all the demographic data needed
df = outcome.merge(patients, on = 'subject_id')
df = df.merge(admissions, on = ['subject_id','hadm_id'])
df = df.merge(icustays, on = ['subject_id','hadm_id'])

In [25]:
#create new demographic variables
df['age'] = df['anchor_age']+(df['admittime'] - df['anchor_year'])/timedelta(days = 365)
df['hours_since_admit'] = (df['chartdate'] - df['admittime']).dt.total_seconds() / 3600

In [26]:
demographic = df[['subject_id', 'hadm_id', 'age', 'first_careunit',
   'gender','admittime', 'dischtime','deathtime', 'chartdate', 'hours_since_admit']]

demographic = demographic.loc[demographic.groupby('hadm_id').hours_since_admit.idxmin()]
##replace with 0 on hours since admit if negative (due to no exact time on procedure bit exact time on admission creating discrepency)
demographic.loc[demographic['hours_since_admit'] < 0, 'hours_since_admit'] = 0

# wrangle chartevents dataset

In [166]:
#chunk in chart events no longer taking latest value as will use LSTM
df = pd.DataFrame()
for i, chartevents in  enumerate(pd.read_csv(path + '/icu/chartevents.csv', chunksize = 1000000, parse_dates = ['charttime'])):
    chartevents_hadm = chartevents[chartevents['hadm_id'].isin(outcome['hadm_id'])]
    chartevents_features = chartevents_hadm[chartevents_hadm['itemid'].isin(features_analysis['itemid'])]
    chartevents_merged = chartevents_features.merge(outcome, on = ['subject_id','hadm_id'])
    chartevents_time = chartevents_merged[(chartevents_merged['charttime'] <= chartevents_merged['obs_window_end'])]
    df_add = chartevents_time.iloc[:,[0,1,3,5,6,7,8]]
    df = pd.concat([df, df_add])
df.to_csv(path + 'small/icu/chartevents_filtered_use.csv', index = False)

/gpfs/commons/home/aelhussein/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/gpfs/commons/home/aelhussein/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## check lab events

In [52]:
d_lab_items = pd.read_csv(path+'hosp/d_labitems.csv')
d_lab_items = d_lab_items[d_lab_items['label'].notna()]

In [199]:
d_lab_items[d_lab_items['label'].str.contains('O2')]

,itemid,label,fluid,category,loinc_code
292,50804,Calculated Total CO2,Blood,Blood Gas,NaN
1120,50815,O2 Flow,Blood,Blood Gas,NaN
1164,50818,pCO2,Blood,Blood Gas,NaN
1165,52040,pCO2,Fluid,Blood Gas,NaN
1166,50830,"pCO2, Body Fluid",Other Body Fluid,Blood Gas,NaN
1219,50821,pO2,Blood,Blood Gas,NaN
1220,52042,pO2,Fluid,Blood Gas,NaN
1221,50832,"pO2, Body Fluid",Other Body Fluid,Blood Gas,NaN
1331,50823,Required O2,Blood,Blood Gas,NaN
1455,51739,Total CO2,Blood,Chemistry,NaN


In [120]:
##features dropped from chartevents
features = ['Alkaline Phosphatase', 'Alanine Aminotransferase (ALT)', 'Ammonia', 
            'Lactate', 'Lactate Dehydrogenase (LD)', 
            'Bilirubin, Total', 'Absolute Neutrophil Count', 
            'Neutrophils', 'C-Reactive Protein', 'D-Dimer']

In [125]:
#extract itemids and save
features_analysis_lab = d_lab_items[d_lab_items['label'].isin(features)]
features_analysis_lab = pd.DataFrame(features_analysis_lab['itemid'])

##find some labels have multiple itemids
features_analysis_lab.merge(d_lab_items, on = 'itemid')

##create a mapper
column_mapper = {52442:50813, 50915:51196, 52551:51196, 51697:51256}

In [ ]:
df = pd.DataFrame()
for i, labevents in  enumerate(pd.read_csv(path + 'hosp/labevents.csv', chunksize = 1000000, parse_dates = ['charttime'])):
    labevents = labevents.drop('hadm_id', axis = 1)
    labevents_subject = labevents[labevents['subject_id'].isin(outcome['subject_id'])]
    labevents_features = labevents_subject[labevents_subject['itemid'].isin(features_analysis_lab['itemid'])]
    labevents_merged = labevents_features.merge(outcome, on = 'subject_id')
    labevents_time = labevents_merged[(labevents_merged['charttime'] <= labevents_merged['obs_window_end'])]
    df_add = labevents_time.loc[:,['subject_id','hadm_id','charttime','itemid','value','valuenum','valueuom']]
    df_add.replace({"itemid": column_mapper}, inplace=True)
    df = pd.concat([df, df_add])

In [131]:
df.to_csv(path + 'small/hosp/labevents_filtered_use.csv', index = False)

# create final dataset

In [27]:
#create the final dataset
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
df_chart = pd.read_csv(path + 'small/icu/chartevents_filtered_use.csv', parse_dates = ['charttime'])
df_lab = pd.read_csv(path + 'small/hosp/labevents_filtered_use.csv', parse_dates = ['charttime'], 
                    dtype={'valunum': np.float64})
df = pd.concat([df_chart, df_lab])

/tmp/ipykernel_420924/3359102325.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chart = pd.read_csv(path + 'small/icu/chartevents_filtered_use.csv', parse_dates = ['charttime'])


In [134]:
df.hadm_id.nunique()

4255

In [142]:
##values that are not numeric
not_numeric = df['itemid'][pd.to_numeric(df['valuenum'], errors='coerce').isnull()].unique()

In [36]:
#create a pivot table where each row is an admission and each column is the feature
df = df.loc[df.groupby(['hadm_id', 'itemid']).charttime.idxmax()]
df_pivot = df.pivot_table(index = 'hadm_id', columns = 'itemid', values = 'valuenum', aggfunc = 'mean')

In [37]:
##filter out features with >20% missing
cols_to_use = df_pivot.columns[df_pivot.isna().sum(axis = 0) / df_pivot.shape[0] < 0.3]
cols_dropped = df_pivot.columns[df_pivot.isna().sum(axis = 0) / df_pivot.shape[0] >= 0.3]
df_pivot = df_pivot.loc[:, cols_to_use]

In [165]:
#features to be used
items[items['itemid'].isin(cols_to_use)]
d_lab_items[d_lab_items['itemid'].isin(cols_to_use)]

,itemid,label,fluid,category,loinc_code
52,50861,Alanine Aminotransferase (ALT),Blood,Chemistry,1742-6
76,50863,Alkaline Phosphatase,Blood,Chemistry,6768-6
231,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
938,50813,Lactate,Blood,Blood Gas,NaN
1090,51256,Neutrophils,Blood,Hematology,761-7


In [152]:
#features dropped
items[items['itemid'].isin(cols_dropped)]
d_lab_items[d_lab_items['itemid'].isin(cols_dropped)]

,itemid,label,fluid,category,loinc_code
31,52075,Absolute Neutrophil Count,Blood,Hematology,751-8
98,50866,Ammonia,Blood,Chemistry,NaN
508,50889,C-Reactive Protein,Blood,Chemistry,1988-5
545,51196,D-Dimer,Blood,Hematology,NaN
942,50954,Lactate Dehydrogenase (LD),Blood,Chemistry,2532-0


In [38]:
##add in the demographic features
df_features = df_pivot.merge(demographic[['hadm_id', 'age', 'first_careunit',
   'gender', 'hours_since_admit']], left_index = True, right_on = 'hadm_id')

In [39]:
df_features.set_index('hadm_id', inplace = True)

In [40]:
#replace location with either medical or surgical unit
locations = {'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU',
       'Trauma SICU (TSICU)':'SICU', 'Coronary Care Unit (CCU)':'MICU',
       'Medical Intensive Care Unit (MICU)':'MICU',
       'Cardiac Vascular Intensive Care Unit (CVICU)':'MICU',
       'Surgical Intensive Care Unit (SICU)':'SICU',
       'Neuro Surgical Intensive Care Unit (Neuro SICU)':'SICU',
       'Neuro Stepdown':'MICU',
       'Neuro Intermediate':'MICU'}

In [41]:
df_features['first_careunit'] = df_features['first_careunit'].replace(locations)

In [42]:
#create one-hot encoding
df_features = pd.get_dummies(df_features, columns = ['first_careunit', 'gender'], drop_first = True)

In [43]:
##remove smaples without much missing
df_samples = df_features.dropna(axis=0, how='any', thresh=0.8*df_features.shape[1], subset=None, inplace=False)

In [44]:
##measure missingness in both groups
missing = pd.DataFrame(np.where(df_samples.isna(), 1, 0), columns = df_samples.columns, index = df_samples.index)
missing = missing.merge(outcome[['hadm_id', 'outcome']], left_index = True, right_on = 'hadm_id')
missing_by_outcome =  missing.groupby('outcome').mean().drop('hadm_id', axis = 1).T

In [45]:
missing_by_outcome

outcome,0,1
50813,0.063392,0.058013
50861,0.105083,0.087020
50863,0.118789,0.089920
50885,0.118789,0.095722
51256,0.023415,0.058013
220045,0.000000,0.000000
220179,0.006853,0.029007
220180,0.006853,0.029007
220210,0.000000,0.000725
220228,0.009138,0.006526


In [57]:
## use random forest imputation for missing values
imputer = MissForest()
df_imputed = imputer.fit_transform(df_samples)

/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed i

Iteration: 0


/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/gpfs/commons/home/aelhussein/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed i

Iteration: 1


In [59]:
df_imputed = pd.DataFrame(df_imputed, columns = df_samples.columns, index = df_samples.index)

In [60]:
gcs = [223900,223901,220739]
df_imputed['gcs'] = df_imputed.loc[:,gcs[0]] + df_imputed.loc[:,gcs[1]] + df_imputed.loc[:,gcs[2]]
df_imputed_cleaned = df_imputed.drop(columns = gcs)

In [61]:
##bucket gcs
bins = [0,5,8,12,15]
labels = [0,1,2,3]
df_imputed_cleaned['gcs_cat'] = pd.cut(df_imputed_cleaned['gcs'], bins, labels = labels)
df_imputed_cleaned.drop('gcs', axis = 1, inplace = True)
df_imputed_cleaned = pd.get_dummies(df_imputed_cleaned, columns = ['gcs_cat'], drop_first = True)
df_imputed_cleaned = df_imputed_cleaned.rename(columns = {'gcs_cat_1': 'gcs_5-8', 'gcs_cat_2': 'gcs_9-12','gcs_cat_3': 'gcs_13-15'})

In [62]:
df_imputed_cleaned.head()

,50813,50861,50863,50885,51256,220045,220179,220180,220210,220228,...,225624,227442,227457,age,hours_since_admit,first_careunit_SICU,gender_M,gcs_5-8,gcs_9-12,gcs_13-15
hadm_id,,,,,,,,,,,,,,,,,,,,,
20009335,1.787350,9.00,79.946833,0.560083,63.200000,77.0,99.0,76.0,22.0,9.7,...,17.0,3.2,173.0,87.691699,2.524722,0.0,1.0,0,0,1
20013839,1.866667,33.75,125.000000,0.200000,84.750000,108.0,111.0,62.0,20.0,6.6,...,12.0,3.7,343.0,65.513044,1.233333,1.0,0.0,0,0,1
20015802,0.900000,35.00,74.000000,0.300000,72.800000,144.0,104.0,74.0,29.0,8.9,...,13.0,4.1,176.0,71.065310,363.883333,0.0,1.0,0,1,0
20020562,1.483333,17.50,80.000000,0.600000,64.500000,98.0,146.0,67.0,21.0,7.1,...,34.0,5.2,111.0,69.596912,291.050000,0.0,1.0,0,0,1
20023225,1.780000,92.00,58.000000,0.700000,86.633333,92.0,129.0,84.0,19.0,10.8,...,47.0,3.9,74.0,92.497308,2.800000,0.0,1.0,0,0,1


In [63]:
df_imputed_cleaned.to_csv('/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/dataset2.csv')

## Extract dicom_ids

In [64]:
df_imputed_cleaned = pd.read_csv('/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0//small/dataset2.csv', index_col = ['hadm_id'])

In [65]:
cxr_use = cxr_use[cxr_use['hadm_id'].isin(df_imputed_cleaned.index)]

In [67]:
#create the naming logic needed to download
cxr_use['folder'] = 'p' + cxr_use['subject_id'].astype(str).str[:2]
cxr_use['sub_folder'] = 'p' + cxr_use['subject_id'].astype(str)
cxr_use['study_folder'] = 's' + cxr_use['study_id'].astype(str)

/tmp/ipykernel_39525/147219020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cxr_use['folder'] = 'p' + cxr_use['subject_id'].astype(str).str[:2]
/tmp/ipykernel_39525/147219020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cxr_use['sub_folder'] = 'p' + cxr_use['subject_id'].astype(str)
/tmp/ipykernel_39525/147219020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [69]:
#create file request
request = 'wget -r -N -c -np --user ahmed-elhussein --ask-password '
files = ''
for i, row in cxr_use.iterrows():
    file = 'https://physionet.org/files/mimic-cxr-jpg/2.0.0/files/{}/{}/{}/{}.jpg '.format(
        row['folder'], row['sub_folder'], row['study_folder'], row['dicom_id'])
    files = files + file
    
request = request+files

## Time series extraction

In [28]:
df_timeseries = df.merge(outcome[['hadm_id', 'obs_window_start','obs_window_end', 'outcome']], on = 'hadm_id')

In [29]:
df_timeseries['window'] = np.nan
##put data into windows
date_windows = [
     df_timeseries['charttime'] <= df_timeseries['obs_window_start'] + timedelta(days = 1),
     (df_timeseries['charttime'] > df_timeseries['obs_window_start'] + timedelta(days = 1)) &
        (df_timeseries['charttime'] <= df_timeseries['obs_window_start'] + timedelta(days = 2)),
     (df_timeseries['charttime'] > df_timeseries['obs_window_start'] + timedelta(days = 2)) &
        (df_timeseries['charttime'] <= df_timeseries['obs_window_start'] + timedelta(days = 3)),
     (df_timeseries['charttime'] > df_timeseries['obs_window_start'] + timedelta(days = 3)) &
        (df_timeseries['charttime'] <= df_timeseries['obs_window_start'] + timedelta(days = 4.5))]

values = [0,1,2,3]

df_timeseries['window'] = np.select(date_windows, values)

In [55]:
lab_freq = (df_timeseries.groupby(['window', 'itemid']).count() / outcome_cohort.shape[0])[
    'subject_id'].sort_values( ascending = False)

In [92]:
lab_freq.loc[0].head(), lab_freq.loc[1].head(),lab_freq.loc[2].head(), lab_freq.loc[3].head(10),

(itemid
 220045    19.172790
 220210    18.989089
 220277    18.831886
 50861     14.191494
 220179    13.598085
 Name: subject_id, dtype: float64,
 itemid
 220045    11.303941
 220210    11.253173
 220277    11.187709
 220179     8.845023
 220180     8.843465
 Name: subject_id, dtype: float64,
 itemid
 220045    8.906925
 220210    8.838121
 220277    8.734803
 220179    6.759964
 220180    6.759074
 Name: subject_id, dtype: float64,
 itemid
 220045    9.824093
 220210    9.742819
 220277    9.448452
 220179    6.703852
 220180    6.702961
 220052    2.747495
 220050    2.733467
 220051    2.733244
 220739    2.452238
 223900    2.448452
 Name: subject_id, dtype: float64)

In [117]:
sequential = [220045,220210,220277, 220739,223900, 223901]
items[items['itemid'].isin(sequential)]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
26,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
33,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
63,220739,GCS - Eye Opening,Eye Opening,chartevents,Neurological,NaN,Text,NaN,NaN
285,223900,GCS - Verbal Response,Verbal Response,chartevents,Neurological,NaN,Text,NaN,NaN
286,223901,GCS - Motor Response,Motor Response,chartevents,Neurological,NaN,Text,NaN,NaN


In [119]:
df_sequential = df_timeseries[df_timeseries['itemid'].isin(sequential)]

In [122]:
#check that sequential data is available
for i in range(4):
    df_window_1 = df_sequential[df_sequential['window'] == i]
    df_pivot_w1 = df_window_1.pivot_table(index = 'hadm_id', columns = 'itemid', values = 'valuenum', aggfunc = 'mean')
    cols_to_use = df_pivot_w1.columns[df_pivot_w1.isna().sum(axis = 0) / df_pivot_w1.shape[0] < 0.1]
    cols_dropped = df_pivot_w1.columns[df_pivot_w1.isna().sum(axis = 0) / df_pivot_w1.shape[0] >= 0.1]
    print(cols_dropped)

Int64Index([], dtype='int64', name='itemid')
Int64Index([], dtype='int64', name='itemid')
Int64Index([], dtype='int64', name='itemid')
Int64Index([], dtype='int64', name='itemid')


In [133]:
df_sequential.to_csv('/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/dataset_sequential.csv', index=False)